# 1-qubit unitary synthesis

In [3]:
import numpy as np
from scipy.stats import unitary_group

u = unitary_group.rvs(2)


global_phase = np.angle(u[0, 0])
u /= np.exp(1j * global_phase)
print(u)

beta = 2 * np.arccos(u[0, 0].real)
alpha = np.angle(1j * u[1, 0] / (np.sin(beta/2)))
gamma = np.angle(u[1, 1] / (np.cos(beta/2))) - alpha
print("alpha =", alpha)
print("beta =", beta)
print("gamma =", gamma)

[[-0.58555973-0.373348j   -0.52248217+0.49471553j]
 [ 0.05019359-0.71778247j  0.66030744+0.21508838j]]
[[ 0.69445585-0.j          0.17458772-0.69803309j]
 [ 0.34356583+0.63221325j -0.67240022+0.17362856j]]
alpha = 2.643804905479036
beta = 1.6062659051267858
gamma = 0.24508578158502914


## Sanity check

In [4]:
def rz(theta):
    return np.diag([1, np.exp(1j * theta)])

def rx(theta):
    return np.cos(theta/2) * np.eye(2) -1j*np.sin(theta/2) * np.array([[0, 1], [1, 0]])
print("Our target matrix:")
print(u)
print("Our circuit:")
print(rz(alpha).dot(rx(beta).dot(rz(gamma))))

[[ 0.69445585+0.j          0.17458772-0.69803309j]
 [ 0.34356583+0.63221325j -0.67240022+0.17362856j]]
[[ 0.69445585-0.j          0.17458772-0.69803309j]
 [ 0.34356583+0.63221325j -0.67240022+0.17362856j]]


# 2-qbit synthesis

The BFGS way !


In [13]:
I =np.eye(2)
def ry(theta):
    return np.cos(theta/2) * I -1j * np.sin(theta/2) * np.array([[0, -1j], [1j, 0]])
    
CNOT = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
SWAP = np.array([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]])
TONC = SWAP.dot(CNOT).dot(SWAP)

In [31]:
def build_matrix(angles):
    mat = np.eye(4, dtype=np.complex128)
    # U1
    mat = np.kron(rz(angles[0]), I).dot(mat)
    mat = np.kron(rx(angles[1]), I).dot(mat)
    mat = np.kron(rz(angles[2]), I).dot(mat)
    # U2
    mat = np.kron(I, rx(angles[3])).dot(mat)
    mat = np.kron(I, rz(angles[4])).dot(mat)
    mat = np.kron(I, rx(angles[5])).dot(mat)
    # 1st CNOT
    mat = TONC.dot(mat)
    # RZ
    mat = np.kron(rz(angles[6]), I).dot(mat)
    # RY
    mat = np.kron(I, ry(angles[7])).dot(mat)
    # 2nd CNOT
    mat = CNOT.dot(mat)
    # RY
    mat = np.kron(I, ry(angles[8])).dot(mat)
     # 3rd CNOT
    mat = TONC.dot(mat)
    # U3
    mat = np.kron(rz(angles[9]), I).dot(mat)
    mat = np.kron(rx(angles[10]), I).dot(mat)
    mat = np.kron(rz(angles[11]), I).dot(mat)
    # U4
    mat = np.kron(I, rx(angles[12])).dot(mat)
    mat = np.kron(I, rz(angles[13])).dot(mat)
    mat = np.kron(I, rx(angles[14])).dot(mat)
    return mat / np.exp(1j*np.angle(mat[0,0]))


In [32]:

u = unitary_group.rvs(4)
global_phase = np.angle(u[0, 0])
u /= np.exp(1j * global_phase)
print(u)


[[ 0.54909212+0.j          0.04182582+0.394408j    0.63831161-0.09569009j
  -0.1994143 -0.29124973j]
 [-0.47386334+0.12778816j -0.45509308-0.19616037j  0.55710292-0.26288514j
  -0.25811272+0.25969344j]
 [ 0.31138189+0.3162686j   0.44729074-0.37552338j  0.27136234+0.0990505j
   0.16326667+0.59314765j]
 [-0.4844001 -0.16122529j  0.36741341+0.34788633j  0.34708337+0.00374806j
   0.60095873-0.04139j   ]]


In [38]:
def distance(angles):
    return np.linalg.norm(u - build_matrix(angles))

from scipy.optimize import minimize

res = minimize(distance, x0=np.random.random(15) * 2 * np.pi, method="BFGS", options={"maxiter": 40000})
print(res)


  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 1.9618956151696055e-07
        x: [ 2.525e+00  5.673e+00 ...  2.222e+00  3.789e+00]
      nit: 131
      jac: [ 1.179e-01 -1.103e-01 ... -7.420e-02  3.355e-02]
 hess_inv: [[ 7.539e-06  9.199e-07 ... -6.664e-07  4.920e-06]
            [ 9.199e-07  6.273e-07 ... -2.624e-06  4.668e-06]
            ...
            [-6.664e-07 -2.624e-06 ...  1.063e-04 -1.714e-04]
            [ 4.920e-06  4.668e-06 ... -1.714e-04  2.810e-04]]
     nfev: 3228
     njev: 201


In [39]:
print("Our target matrix:")
print(u)
print("Our circuit:")
print(build_matrix(res.x))

[[ 0.54909214-9.83716036e-17j  0.04182585+3.94408062e-01j
   0.63831157-9.56901084e-02j -0.19941427-2.91249705e-01j]
 [-0.47386332+1.27788088e-01j -0.45509303-1.96160394e-01j
   0.55710297-2.62885117e-01j -0.25811279+2.59693427e-01j]
 [ 0.31138188+3.16268583e-01j  0.44729075-3.75523349e-01j
   0.27136235+9.90505454e-02j  0.1632667 +5.93147670e-01j]
 [-0.48440015-1.61225254e-01j  0.36741341+3.47886336e-01j
   0.34708335+3.74806848e-03j  0.6009587 -4.13900465e-02j]]
[[ 0.54909212+0.j          0.04182582+0.394408j    0.63831161-0.09569009j
  -0.1994143 -0.29124973j]
 [-0.47386334+0.12778816j -0.45509308-0.19616037j  0.55710292-0.26288514j
  -0.25811272+0.25969344j]
 [ 0.31138189+0.3162686j   0.44729074-0.37552338j  0.27136234+0.0990505j
   0.16326667+0.59314765j]
 [-0.4844001 -0.16122529j  0.36741341+0.34788633j  0.34708337+0.00374806j
   0.60095873-0.04139j   ]]


# CNOT circuits

## Gaussian elimination

In [62]:
def gaussian_elimination(table):
    table = table.copy()
    output = []
    for i in range(table.shape[1]):
        pivot = None
        for j in range(i, table.shape[0]):
            if table[j, i]:
                pivot = j
                break
        if pivot != i:
            table[[i, pivot]] = table[[pivot, i]]
            output.append((i, pivot))
            output.append((pivot, i))
            output.append((i, pivot))
        for j in range(table.shape[0]):
            if table[j, i] and i != j:
                table[j] ^= table[i]
                output.append((i, j))
    return output

## Greedy Gaussian elimination

In [63]:
def lu_decomposition(matrix):
    P = np.eye(matrix.shape[0], dtype=np.byte)
    U = matrix.copy()
    L = np.eye(matrix.shape[0], dtype=np.byte)
    ops = []
    for i in range(matrix.shape[0]):
        pivot = None
        for j in range(i, matrix.shape[0]):
            if U[j, i]:
                pivot = j
                break
        if pivot != i:
            P[[i, pivot]] = P[[pivot, i]]
            U[[i, pivot]] = U[[pivot, i]]
            ops.extend(((i, pivot), (pivot, i), (i, pivot)))
        for j in range(i + 1, matrix.shape[0]):
            if U[j, i]:
                U[j] ^= U[i]
                ops.append((i, j))
    for a, b in reversed(ops):
        L[b] ^= L[a]
    return P, U, P @ L


def greedy_ge(triangle):
    """ Reduction of a lower triangular """
    
    triangle = triangle.copy()
    output = []
    for i in range(triangle.shape[1]):
        while any(triangle[i+1:, i]):
            candidates = np.nonzero(triangle[:, i])[0]
            candidates = [(c, t) for i, c in enumerate(candidates) for t in candidates[i+1:]]
            def score(cnot):
                c, t = cnot
                s = 0
                for value in (triangle[c] & triangle[t]):
                    if value:
                        s += 1
                        continue
                    break
                return s
            best_pick = max(candidates, key=score)
            output.append(best_pick)
            triangle[best_pick[1]] ^= triangle[best_pick[0]]
    return output

def rev_args(cnots):
    return [(b,a) for a, b in cnots]

def greedy_gaussian_elimination(table):
    p, u, l = lu_decomposition(table)
    circuit_u = rev_args(greedy_ge(u.transpose()))
    circuit_l = list(reversed(greedy_ge(l)))
    output = circuit_u + circuit_l
    return output

## Benchmarks!


In [64]:
def gen_random_table(n):
    mat = np.eye(n, dtype=np.byte)
    for _ in range(n * n):
        a, b = np.random.choice(range(n), size=2, replace=False)
        mat[b] ^= mat[a]
    return mat

In [78]:
A = gen_random_table(40)

circuit_ge = gaussian_elimination(A)
circuit_gge = greedy_gaussian_elimination(A)
print(len(circuit_ge))
print(len(circuit_gge))

787
711
